Imports of all libraries and variables that will be used

In [ ]:
import cv2 as cv
import depthai as dai
import numpy as np
import blobconverter
from scipy.spatial.transform import Rotation
import json

rgb_json = 'intrinsics_rgb.json' # Input the path to the intrinsic_rgb json file
robot_file = 'path_to_robot_file'

Helper functions

In [ ]:
def quat_to_R(q):
    x,y,z,w = q
    # normalize
    s = np.linalg.norm([w,x,y,z])
    w,x,y,z = w/s, x/s, y/s, z/s
    R = np.array([
        [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
        [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
        [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
    ])
    return R

# SHOUTOUT https://github.com/RealManRobot/hand_eye_calibration/blob/main/compute_to_hand.py
def convert(x ,y ,z, rotation_matrix, translation_vector):
    obj_camera_coordinates = np.array([x, y, z])
    T_camera_to_base_effector = np.eye(4)
    T_camera_to_base_effector[:3, :3] = rotation_matrix
    T_camera_to_base_effector[:3, 3] = translation_vector.reshape(3)

    # Compute the pose of the object against the base
    obj_camera_coordinates_homo = np.append(obj_camera_coordinates, [1])  # Convert object coordinates to homogeneous coordinates
    obj_base_effector_coordinates_homo = T_camera_to_base_effector.dot(obj_camera_coordinates_homo)
    obj_base_coordinates = obj_base_effector_coordinates_homo[:3]  

    rot_matrix_homo = T_camera_to_base_effector[:3, :3]
    quaternion = Rotation.from_matrix(rot_matrix_homo).as_quat()

    return list(obj_base_coordinates), list(quaternion)

def get_rtvec_t2c(intrinsincs_file):
    with open(intrinsincs_file, 'r') as intrinsic_data:
        data = json.load(intrinsic_data)
        rvecs = [[val[0] for val in vec] for vec in data["rvecs"]]
        tvecs = [[val[0] * 10 for val in vec] for vec in data["tvecs"]] # We multiply with 10 to change the tvecs from cm to mm
    return rvecs, tvecs   

def get_robot_pos_quat(robot_file):
    pass

def camera2robot_calib(position_robot, quat_robot, rotation_vec_t2c, translation_t2c):
    R_g2b, t_g2b = [], []
    for pos, quat in zip(position_robot, quat_robot): #position robot list of XYZ for each position of calibration and quat is its quaternion
        R_b2g = quat_to_R(quat)                                 # (3,3)
        t_b2g = np.asarray(pos, np.float64).reshape(3,1)  #  (3,1)
        # invert to gripper->base
        R_gb = R_b2g.T
        t_gb = - R_gb @ t_b2g
        R_g2b.append(R_gb)
        t_g2b.append(t_gb)

    R_t2c = [cv.Rodrigues(np.array(rv, np.float64))[0] for rv in rotation_vec_t2c]
    t_t2c = [np.array(tv, np.float64).reshape(3,1)     for tv in translation_t2c]

    R_cam2gripper, t_cam2gripper = cv.calibrateHandEye(
        R_gripper2base=R_g2b, t_gripper2base=t_g2b,
        R_target2cam=R_t2c,   t_target2cam=t_t2c,
        method=cv.CALIB_HAND_EYE_DANIILIDIS
    )
    return R_cam2gripper, t_cam2gripper

def json_converter(coordinates, quaternion):
    data = [coordinates, quaternion] # We store the coordinates and quaternion in a list so that it can be easily accessible.

    # The layout/structure of the JSON file
    cup = {
        "id":"cup_1",
        "status": "Available",
        "position":{
            "x": data[0][0],
            "y": data[0][1],
            "z": data[0][2]
        },
        "orientation":{
            "q1": float(data[1][0]),
            "q2": float(data[1][1]),
            "q3": float(data[1][2]),
            "q4": float(data[1][3])
        },
        "approach_position":{
            "x": data[0][0],
            "y": data[0][1],
            "z": data[0][2]
        }
    }

    cup_data = {"cups": cup}

    with open("cups.json", "w") as f:
        json.dump(cup_data, f, indent=2)


Camera setup

In [ ]:
# Create DepthAI pipeline
pipeline = dai.Pipeline()

# Define camera nodes
cam_rgb   = pipeline.create(dai.node.ColorCamera)
mono_left  = pipeline.create(dai.node.MonoCamera)
mono_right = pipeline.create(dai.node.MonoCamera)
stereo     = pipeline.create(dai.node.StereoDepth)
detection_nn = pipeline.create(dai.node.YoloSpatialDetectionNetwork)  # Spatial NN for YOLO

# Define XLink outputs for streaming frames and detections to host
xout_rgb   = pipeline.create(dai.node.XLinkOut)
xout_depth = pipeline.create(dai.node.XLinkOut)
xout_nn    = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
xout_depth.setStreamName("depth")
xout_nn.setStreamName("detections")

# Camera configuration (RGB camera)
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setPreviewSize(416, 416)  # neural network input size for TinyYOLOv4
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

# Mono cameras (for depth)
mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
mono_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)

# Stereo depth configuration
stereo.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)
stereo.setDepthAlign(dai.CameraBoardSocket.RGB)        # Align depth map to RGB camera perspective:contentReference[oaicite:25]{index=25}
stereo.setOutputSize(mono_left.getResolutionWidth(), 
                     mono_left.getResolutionHeight())
stereo.setSubpixel(True)  # improve depth precision

# Download and set neural network model (Tiny-YOLOv4 COCO 416x416) <------- THIS CAN BE WHATEVER MODEL THAT EXISTS IN THE DEPTHAI ZOO
blob_path = blobconverter.from_zoo(name="yolov4_tiny_coco_416x416", 
                                   zoo_type="depthai", 
                                   shaves=6)
detection_nn.setBlobPath(str(blob_path))
detection_nn.setConfidenceThreshold(0.5)
detection_nn.input.setBlocking(False)
detection_nn.setBoundingBoxScaleFactor(0.5)  # extra area for depth averaging
detection_nn.setDepthLowerThreshold(100)     # 100 mm -> 10 cm minimum distance:contentReference[oaicite:26]{index=26}
detection_nn.setDepthUpperThreshold(5000)    # 5000 mm -> 5 m maximum distance:contentReference[oaicite:27]{index=27}

# YOLO-specific network settings (for COCO Tiny-YOLOv4 416x416)
detection_nn.setNumClasses(80)
detection_nn.setCoordinateSize(4)
detection_nn.setAnchors([10,14, 23,27, 37,58, 81,82, 135,169, 344,319])       # from model config:contentReference[oaicite:28]{index=28}
detection_nn.setAnchorMasks({ "side26": [1,2,3], "side13": [3,4,5] })         # YOLOv4-tiny masks:contentReference[oaicite:29]{index=29}
detection_nn.setIouThreshold(0.5)

# Link nodes: RGB -> Neural Network, Mono -> StereoDepth, Depth -> Neural Network
cam_rgb.preview.link(detection_nn.input)
mono_left.out.link(stereo.left)
mono_right.out.link(stereo.right)
stereo.depth.link(detection_nn.inputDepth)

# Link NN outputs to XLink outputs
detection_nn.passthrough.link(xout_rgb.input)      # passthrough RGB frames (frames that went into NN)
detection_nn.passthroughDepth.link(xout_depth.input)  # aligned depth frames
detection_nn.out.link(xout_nn.input)              # detection outputs (bounding boxes + coordinates)

# Initialize the device and pipeline
with dai.Device(pipeline) as device:
    # Output queues to retrieve frames and detections
    q_rgb   = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    q_depth = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
    q_det   = device.getOutputQueue(name="detections", maxSize=4, blocking=False)

    # Get label names for COCO classes (for display purposes)
    label_map = [
        "person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat",
        "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
        "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella",
        "handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite",
        "baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle",
        "wine glass","cup","fork","knife","spoon","bowl","banana","apple","sandwich",
        "orange","broccoli","carrot","hot dog","pizza","donut","cake","chair","sofa",
        "pottedplant","bed","diningtable","toilet","tvmonitor","laptop","mouse","remote",
        "keyboard","cell phone","microwave","oven","toaster","sink","refrigerator","book",
        "clock","vase","scissors","teddy bear","hair drier","toothbrush"
    ]

Code that needs to be ran once before running the continous loop

In [ ]:
rotation_vec_t2c, translation_t2c = get_rtvec_t2c(rgb_json)
position_robot, quat_robot = get_robot_pos_quat(robot_file)

R_cam2base, t_cam2base = camera2robot_calib(position_robot, quat_robot, rotation_vec_t2c, translation_t2c)

Continously running loop

In [ ]:
while True:
    in_rgb   = q_rgb.get()      # latest RGB frame
    in_depth = q_depth.get()    # latest depth frame (aligned to RGB)
    in_dets  = q_det.get()      # latest detection results

    frame = in_rgb.getCvFrame()               # OpenCV BGR frame from color camera
    depth_frame = in_depth.getFrame()         # depth data in millimeters
    detections = in_dets.detections           # list of spatial detections

    # Iterate over detections and draw bounding boxes and labels (SET TO ONLY DISPLAY CUPS AT THE MOMENT)
    for det in detections:
        
        # Determine label text to display
        label = str(det.label)
        if det.label < len(label_map):
            label = label_map[det.label]
        conf  = int(det.confidence * 100)  # confidence percentage

        if label == "cup":
            # Get bounding box coordinates (normalized 0..1 from NN, convert to pixel coords)
            x1 = int(det.xmin * frame.shape[1])
            y1 = int(det.ymin * frame.shape[0])
            x2 = int(det.xmax * frame.shape[1])
            y2 = int(det.ymax * frame.shape[0])

            # Draw rectangle on RGB frame
            cv.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

            # Draw label and confidence
            cv.putText(frame, f"{label} ({conf}%)", (x1+5, y1+20),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

            # Draw spatial coordinates (X, Y, Z in mm)
            coords = det.spatialCoordinates  # spatial coordinates relative to camera
            cv.putText(frame, f"X: {int(coords.x)} mm", (x1+5, y1+35),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            cv.putText(frame, f"Y: {int(coords.y)} mm", (x1+5, y1+50),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            cv.putText(frame, f"Z: {int(coords.z)} mm", (x1+5, y1+65),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

            coordinates, quaternion = convert(coords.x,coords.y,coords.z, R_cam2base, t_cam2base) # Convert from camera coordinates to robot coordinates.
            json_converter(coordinates, quaternion)
            

    # Show the frames in windows
    cv.imshow("RGB", frame)

    # Exit on 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cv.destroyAllWindows()